In [38]:
from collections import deque
import random

In [2]:
GOAL = [[1,2,3], [8,0,4], [7,6,5]]  # solution state as 2d arr (0 = blank)
N = 3                               # size of state space = NxN

row_moves = [0, 0, -1, 1]    # L/R/U/D values for row movements
col_moves = [-1, 1, 0, 0]    # L/R/U/D values for col movements

In [37]:
# Puzzle state space, as 2d arr = [[0,1,2], [3,4,5], [6,7,8]]
class EightPuzzle:
  def __init__(self, state, x, y, level):
    self.state = state    # state space represented as 2d array
    self.x = x            # x (row) coord of blank tile (zero)
    self.y = y            # y (col) coord of blank tile (zero)
    self.level = level    # depth level (ie: # moves from root)

In [49]:
# Randomize grid with values [0,8], return grid & xy coords of blank tile (0)
def randomize_puzzle_state():
  tile_values = list(range(9))  # list of integers [0,8], 0 = blank tile
  random.shuffle(tile_values)   # shuffle tile positions

  puzzle_state = [tile_values[i : i+3] for i in range(0, 9, 3)]  # convert to grid

  for x in range(3):
    for y in range(3):               # search each col in earch row
      if puzzle_state[x][y] == 0:    # for blank tile (value = 0)
        return puzzle_state, x, y    # return grid & xy coords of blank tile

In [4]:
# Check if current state matches solution state
def is_goal(state):
  return state == GOAL

In [5]:
# Validate move is legal
def is_legal_move(x, y):
  return 0 <= x < N and 0 <= y < N

In [6]:
# Print 2d array state space as NxN grid
def to_grid(state):
  for row in state:
    print(' '.join(map(str, row)))
  print("--------")

In [17]:
# Swap blank tile using row_moves/col_moves
def update_state(current, curr_x, curr_y, move_x, move_y):
  new_state = [row[:] for row in current.state]  # copy state by row then swap coords of blank (0) tile & neighbor
  new_state[current.x][current.y], new_state[move_x][move_y] = new_state[move_x][move_y], new_state[current.x][current.y]
  return new_state


In [21]:
# DFS
def dfs_solver(state, x, y):
  STACK = []     # LIFO stack for exploring branch depths iteratively
  SEEN = set()   # re-init per instance, stores seen states to prevent cycling

  STACK.append(EightPuzzle(state, x, y, 0))  # push root to stack
  SEEN.add(tuple(map(tuple, state)))         # add to set of seen nodes

  while STACK:              # while nodes (states) remain on stack
    current = STACK.pop()   # pop the most recently added node

    print(f"Depth = {current.level}")  # print current depth
    to_grid(current.state)             # print state as NxN grid

    if is_goal(current.state):
      print(f"Solution found at depth {current.level}.")  # solution state reached
      return  # exit

    for i in range(4):                   # move blank tile
      move_x = current.x + row_moves[i]  # new x = current + L/R/U/D
      move_y = current.y + col_moves[i]  # new y = current + L/R/U/D

      if is_legal_move(move_x, move_y):  # validate move & update state
        new_state = update_state(current.state, current.x, current.y, move_x, move_y)

        state_tuple = tuple(map(tuple, new_state))  # convert to tuples

        if state_tuple not in SEEN:
          SEEN.add(state_tuple)   # add visited state to set, push state to stack
          STACK.append(EightPuzzle(new_state, move_x, move_y, current.level+1))

  print("No solution found.")  # all possible moves expired

In [20]:
def bfs_solver(state, x, y):
  QUE = deque()  # FIFO queue for searching complete tiers
  SEEN = set()   # re-init per instance, prevents cycling

  QUE.append(EightPuzzle(state, x, y, 0))  # init root & enqueue
  SEEN.add(tuple(map(tuple, state)))       # add tuple to visited nodes

  while QUE:                  # while states remain in queue
    current = QUE.popleft()   # pop firstmost node

    print(f"Depth = {current.level}")  # print current depth
    to_grid(current.state)             # print current state as NxN grid

    if is_goal(current.state):
      print(f"Solution found at depth {current.level}")  # solution state reached
      return  # exit

    for i in range(4):                   # move blank tile
      move_x = current.x + row_moves[i]  # new x = current + L/R/U/D
      move_y = current.y + col_moves[i]  # new y = current + L/R/U/D

      if is_legal_move(move_x, move_y):  # validate move & update state
        new_state = update_state(current.state, current.x, current.y, move_x, move_y)

        state_tuple = tuple(map(tuple, new_state))     # updated state space

        if state_tuple not in SEEN:
          SEEN.add(state_tuple)   # add visited state to set, push state to stack
          QUE.append(EightPuzzle(new_state, move_x, move_y, current.level+1))

  print("No solution found.")  # all possible moves expired

In [47]:
# Helper method to display user menu
def get_menu_choice(menu):
  while True:   # loop input prompt until valid input is returned
    print(menu)
    choice = input("\nEnter a menu option: ").strip()  # get input, no spaces
    if choice == '1' or choice == '2' or choice == '3' or choice == '4':
      return choice
    print("Invalid input. Enter a single number from the menu: 1, 2, 3, or 4.")


In [52]:
# Driver method with ui menu
def main():
  menu = """\n======[8 PUZZLE MAIN MENU]======
  1. Generate a New 8-Puzzle Board
  2. Solve via BFS (Breadth-First Search)
  3. Solve via DFS (Depth-First Search - Deepens Iteratively)
  4. Exit"""

  puzzle = None      # default null (prevent BFS or DFS)
  x, y = None, None  # default null

  while True:
    choice = get_menu_choice(menu)  # display menu prompt & return valid input

    # Menu 1: Generate new 8-Puzzle Board
    if choice == '1':
      puzzle, x, y = randomize_puzzle_state()  # create randomized board

    # Menu 2: Solve via BFS
    if choice == '2':
      if not puzzle: print("Requires an 8-Puzzle board! Use menu option 1 first.")
      else: bfs_solver(puzzle, x, y)  # call bfs, pass state & blank tile coords

    # Menu 3: Solve via DFS
    if choice == '3':
      if not puzzle: print("Requires an 8-Puzzle board! Use menu option 1 first.")
      else: dfs_solver(puzzle, x, y)  # call dfs, pass state & blank tile coords

    # Menu 4: Exit
    if choice == '4':
      print("Exiting program... Goodbye!")

In [31]:
# Tester
if __name__ == '__main__':
  root = [[1,2,3], [8,0,4], [7,6,5]]   # root state space
  x, y = 1, 1                          # blank tile coords

  print("Solving 8-Puzzle via Iterative DFS in O(n!)...\n")
  dfs_solver(root, x, y)

  print("\n=============================================")

  print("\nSolving 8-Puzzle via BFS in O(n!)...\n")
  bfs_solver(root, x, y)

Solving 8-Puzzle via Iterative DFS in O(n!)...

Depth = 0
1 2 3
8 0 4
7 6 5
--------
Solution found at depth 0.


Solving 8-Puzzle via BFS in O(n!)...

Depth = 0
1 2 3
8 0 4
7 6 5
--------
Solution found at depth 0
